In [2]:
!pip install emoji --upgrade

  Using cached emoji-0.6.0.tar.gz (51 kB)
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49714 sha256=68cf2cb4c1708447812c17a7c312f5825b912dfba0bcc2c13eedfed1b708cdd3
  Stored in directory: /Users/shokazari/Library/Caches/pip/wheels/4e/bf/6b/2e22b3708d14bf6384f862db539b044d6931bd6b14ad3c9adc
Successfully built emoji


In [19]:
# -*- coding: utf-8 -*-
import tweepy
import emoji
import os, config
import json
import argparse, re

import numpy as np
import pandas as pd

def emoji_remove(stc_str):
     return ''.join(c for c in stc_str if c not in emoji.UNICODE_EMOJI)


CK = config.CONSUMER_KEY
CS = config.CONSUMER_SECRET
AT = config.ACCESS_TOKEN
ATS = config.ACCESS_TOKEN_SECRET

auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AT, ATS)
api = tweepy.API(auth, wait_on_rate_limit=True)

#DataFrame
tweet_df = pd.DataFrame(columns=['search_word', 'created_at', 'user_id', 'tweet_id', 'like', 'retweet', 'tweet'])

search_word = '東京理科 -filter:retweets -filter:replies'

for i in range(1, 50):

    if i == 1:
        tweets = api.search(q=search_word, count=10, lang='ja', tweet_mode='extended') #文字数が多いツイートを省略しないようにする.
    else:
        tweets = api.search(q=search_word, count=10, lang='ja', tweet_mode='extended', max_id = next_max_id - 1)

    #tweets = api.search(q=search_word, lang='ja', tweet_method='extended')
    for tweet in tweets:
        search_word = search_word
        created_at = tweet.created_at
        user_id = tweet.user.id_str
        tweet_id = tweet.id_str
        like = tweet.favorite_count
        retweet = tweet.retweet_count
        tweet = emoji_remove(tweet.full_text)

        result_info = pd.Series([search_word, created_at, user_id, tweet_id, like, retweet, tweet], index=tweet_df.columns)
        next_max_id = tweets[-1].id

        if re.search('東京理科', tweet) == None:
            pass
        else:
            tweet_df =  tweet_df.append(result_info, ignore_index=True)


tweet_df.head(100)

,search_word,created_at,user_id,tweet_id,like,retweet,tweet
0,東京理科 -filter:retweets -filter:replies,2020-09-22 08:25:25,1078809288,1308321491684003841,0,0,人気企業100社への就職率ランキング３２．５％ 慶應義塾 ２２．０％ 名古屋大２１．７％ 京...
1,東京理科 -filter:retweets -filter:replies,2020-09-22 03:22:18,2987760439,1308245209147305986,0,0,【4月番発情報】 #番発情報\n12日：慶應・日大商\n13日：明治学院アナ\n19日：東京...
2,東京理科 -filter:retweets -filter:replies,2020-09-21 17:25:25,1078809288,1308094996902371329,0,0,人気企業100社への就職率ランキング３２．５％ 慶應義塾 ２２．０％ 名古屋大２１．７％ 京...
3,東京理科 -filter:retweets -filter:replies,2020-09-21 03:22:18,2987760439,1307882822066003969,0,0,【4月番発情報】 #番発情報\n12日：慶應・日大商\n13日：明治学院アナ\n19日：東京...


In [54]:
# -*- coding: utf-8 -*-

import tweepy
import pandas as pd
import datetime
from time import sleep
import argparse
import config
import emoji

'''
#コマンドラインオプションの設定
parser = argparse.ArgumentParser(description='Twitter APIでツイートを取得')
parser.add_argument('-q', help='検索クエリ')
parser.add_argument('--cnt', default=100, help='取得件数、デフォルト100件')
args = parser.parse_args()
'''

search_words = "東京理科大学留年"

#APIの認証
CK = config.CONSUMER_KEY
CS = config.CONSUMER_SECRET
AT = config.ACCESS_TOKEN
ATS = config.ACCESS_TOKEN_SECRET


auth = tweepy.OAuthHandler(CK, CS)
auth.set_access_token(AT, ATS)
api = tweepy.API(auth)

#ツイート取得
tweet_data = []

try:
    responses = api.search(q=search_words, count=1, result_type="result_type", lang="ja", tweet_mode='extended')
except tweepy.error.TweepError as tweepy_error:
    print(tweepy_error)

next_max_id = responses[-1].id

i = 0

while True:
    print('検索ページ' + str(i))

    try:
        responses = api.search(q=search_words, count=100, max_id = next_max_id-1, lang="ja", tweet_mode='extended')
        if len(responses) > 0:
            next_max_id = responses[-1].id
        else:
            print("これ以上は取得できません。検索ツイートが７日以上か、検索のボリュームが少なすぎます。")
            break

        for tweets in responses:
            user_mentions = []
            if tweets.entities['user_mentions']:
                for name in tweets.entities['user_mentions']:
                    user_mentions.append(name['screen_name'])

            mention_names = ','.join(user_mentions)

            tweet_data.append([tweets.created_at.strftime('%Y/%m/%d'),
                           tweets.id, tweets.user.name, tweets.user.screen_name,
                           tweets.full_text.replace('\n', ''), tweets.favorite_count,
                           tweets.retweet_count, tweets.user.followers_count, tweets.user.friends, mention_names])
    except tweepy.error.TweepError as tweepy_error:
        print(tweepy_error)
        break
    except tweepy.error.RateLimitError as limit_error:
        print("検索の上限に達しました")
        break

    i += 1
    if (i == 10):
        print("上限に達しました。上限は１５分後にリセットされます。")
        break
print ("Finish Search")

columns = ["投稿日", "ID", "User_name", "Screen_name", "text", "favorite", "retweet_count", "followers", "follows", "user_mentions"]

df = pd.DataFrame(tweet_data)
df.columns = columns

df

検索ページ0
検索ページ1
これ以上は取得できません。検索ツイートが７日以上か、検索のボリュームが少なすぎます。
Finish Search


,投稿日,ID,User_name,Screen_name,text,favorite,retweet_count,followers,follows,user_mentions
0,2020/09/22,1308241569665839104,かっつー,physics_AKI,我らが立教大学理学部物理学科は天下の東京理科大学をも上回る留年率を誇ります。,4,0,117,<bound method User.friends of User(_api=<tweep...,
1,2020/09/22,1308219915401388032,明星大学に通う兄を持つ毒舌な妹,meisei_sister,東京理科大学はあまりの留年率の高さから「五年制大学」と言われている。一部の頭のいい人は、飛び...,0,0,987,<bound method User.friends of User(_api=<tweep...,
2,2020/09/21,1308018607134183425,おじょえ留年bot,ozyoe_bot,小説「坊ちゃん」の時代の東京理科大学、もとい東京物理学校にはそもそも留年という制度がなくて進...,0,0,135,<bound method User.friends of User(_api=<tweep...,
3,2020/09/19,1307321481890639872,明星大学に通う兄を持つ毒舌な妹,meisei_sister,東京理科大学はあまりの留年率の高さから「五年制大学」と言われている。一部の頭のいい人は、飛び...,0,0,987,<bound method User.friends of User(_api=<tweep...,
4,2020/09/19,1307157345802174465,だいせん@けんきうかつどう,Totti_D1062,あなたは「東京理科大学さん」タイプ！ https://t.co/dbvoWLPEnj #大学...,1,0,1042,<bound method User.friends of User(_api=<tweep...,
5,2020/09/18,1306800649485447168,五十鈴川ゆき快速急行,special_rapid_,【東京理科大学】留年しました。 https://t.co/e5MQHWDXri @YouTu...,0,0,2066,<bound method User.friends of User(_api=<tweep...,YouTube
6,2020/09/18,1306792486401126400,五十鈴川ゆき快速急行,special_rapid_,【留年しやすいって本当？】東京理科大学の秘密に迫る。 https://t.co/GDSBHb...,0,0,2066,<bound method User.friends of User(_api=<tweep...,YouTube
7,2020/09/14,1305571832028749824,YSK,poki_ta,【悲報】東京理科大学「ぼっち辞めますか？大学辞めますか？」 https://t.co/U3X...,0,0,74,<bound method User.friends of User(_api=<tweep...,news23vip
8,2020/09/14,1305541083581964288,雅@…,miyavihiroshi,東京理科大学の数学科ともなると…演劇にのめり込みつつ留年せずに4年で卒業はなかなかシンドイは...,0,0,459,<bound method User.friends of User(_api=<tweep...,
9,2020/09/14,1305396278608539649,明星大学に通う兄を持つ毒舌な妹,meisei_sister,東京理科大学はあまりの留年率の高さから「五年制大学」と言われている。一部の頭のいい人は、飛び...,1,0,987,<bound method User.friends of User(_api=<tweep...,
